# Lossy-VAE Demo for Google Colab & Kaggle

This notebook demonstrates how to use the Lossy-VAE project for image compression on Google Colab or Kaggle.

## Setup Instructions

**For Google Colab:**
1. Upload the `DataCompression` folder to `/content/`
2. Make sure GPU is enabled (Runtime → Change runtime type → GPU)

**For Kaggle:**
1. Upload the `DataCompression` folder as a dataset
2. Make sure GPU accelerator is selected
3. Update the dataset path in the setup cell below



In [ ]:
import os
import sys
from pathlib import Path

def detect_platform():
    if 'COLAB_GPU' in os.environ:
        return 'colab'
    elif 'KAGGLE_KERNEL_TYPE' in os.environ:
        return 'kaggle'
    else:
        return 'unknown'

platform = detect_platform()
print(f"Platform detected: {platform}")

if platform == 'colab':
    project_dir = Path('/content/DataCompression/lossy-vae')
    datasets_root = Path('/content/datasets')
elif platform == 'kaggle':
    project_dir = Path('/kaggle/working/DataCompression/lossy-vae')
    datasets_root = Path('/kaggle/input/datasets')
    # If you uploaded as a dataset, copy it to working directory
    if not project_dir.exists():
        import shutil
        input_path = Path('/kaggle/input')
        # Find the dataset folder (you may need to adjust this)
        for item in input_path.iterdir():
            if 'datacompression' in item.name.lower():
                shutil.copytree(item / 'DataCompression', 
                              '/kaggle/working/DataCompression', 
                              dirs_exist_ok=True)
                break
else:
    project_dir = Path('.')
    datasets_root = Path('./datasets')

print(f"Project directory: {project_dir}")
print(f"Datasets directory: {datasets_root}")

if not project_dir.exists():
    print(f"ERROR: Project directory not found at {project_dir}")
    print("Please upload the project files first!")
else:
    os.chdir(project_dir)
    print(f"Changed to directory: {os.getcwd()}")



## Install Dependencies



In [ ]:
import subprocess

print("Fixing numpy compatibility issue...")
print("Installing compatible numpy version (1.26.4)...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '--force-reinstall', 'numpy==1.26.4'], check=False)

dependencies = ['tqdm', 'timm', 'compressai']

print("\nInstalling dependencies...")
for dep in dependencies:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', dep], check=False)

print("\nChecking PyTorch...")
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("\nVerifying numpy version...")
import numpy as np
print(f"NumPy version: {np.__version__}")

print("\nInstalling project...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=False)

print("\n✅ Setup complete!")
print("\n⚠️  IMPORTANT: Restart runtime now (Runtime → Restart runtime)")
print("   Then run the next cell to load the model.")



## ⚠️ RESTART RUNTIME NOW!

**After running the setup cell above, you MUST restart the runtime:**

- **Colab**: `Runtime` → `Restart runtime`
- **Kaggle**: `Session` → `Restart Session`

This is required for numpy version changes to take effect.



## Load Pre-trained Model



In [ ]:
import numpy as np
import torch

print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Check if numpy version is compatible
if int(np.__version__.split('.')[0]) >= 2:
    print("\n⚠️  WARNING: NumPy 2.x detected! This will cause compatibility issues.")
    print("\n🔧 Fix steps:")
    print("1. Run this in a new cell:")
    print("   !pip install --force-reinstall numpy==1.26.4")
    print("2. Restart runtime: Runtime → Restart runtime")
    print("3. Run this cell again")
    raise ValueError("NumPy 2.x is incompatible. Please install numpy 1.26.4 and restart runtime.")

print("✅ NumPy version looks compatible")

try:
    # Import models to register them
    from lvae.models import qarv, qresvae, rd
    from lvae import get_model
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\nUsing device: {device}")
    
    model_name = 'qarv_base'
    print(f"Loading model: {model_name}")
    
    model = get_model(model_name, pretrained=True)
    model = model.to(device)
    model.eval()
    model.compress_mode(True)
    
    print("✅ Model loaded successfully!")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
except ValueError as e:
    if "numpy.dtype size changed" in str(e):
        print("\n❌ NumPy compatibility issue detected!")
        print("\n🔧 Fix steps:")
        print("1. Run this command in a new cell:")
        print("   !pip install --force-reinstall numpy==1.26.4")
        print("2. Restart runtime: Runtime → Restart runtime")
        print("3. Run this cell again")
        raise
    else:
        raise
except Exception as e:
    print(f"\n❌ Error: {e}")
    if "numpy" in str(e).lower():
        print("\n🔧 If this is a numpy error, try:")
        print("1. !pip install --force-reinstall numpy==1.26.4")
        print("2. Restart runtime")
        print("3. Run this cell again")
    raise
    else:
        raise



## Example 1: Compress and Decompress an Image

Let's compress an image and then decompress it to see the results.



In [ ]:
from PIL import Image
import torchvision.transforms.functional as tvf
from torchvision.utils import save_image
import matplotlib.pyplot as plt

if platform == 'colab':
    from google.colab import files
    from IPython.display import display, Image as IPImage

test_image_path = project_dir / 'images' / 'collie128.png'

if not test_image_path.exists():
    print(f"Test image not found at {test_image_path}")
    print("Please upload an image or use a different path")
else:
    print(f"Loading image from: {test_image_path}")
    
    img = Image.open(test_image_path)
    print(f"Image size: {img.size}")
    
    compressed_path = '/tmp/compressed.bits'
    
    model.compress_file(str(test_image_path), compressed_path)
    
    file_size = Path(compressed_path).stat().st_size
    original_size = test_image_path.stat().st_size
    compression_ratio = original_size / file_size
    
    print(f"\nCompression stats:")
    print(f"Original size: {original_size:,} bytes")
    print(f"Compressed size: {file_size:,} bytes")
    print(f"Compression ratio: {compression_ratio:.2f}x")
    
    reconstructed = model.decompress_file(compressed_path)
    
    save_path = '/tmp/reconstructed.png'
    save_image(reconstructed, save_path)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    recon_img = Image.open(save_path)
    axes[1].imshow(recon_img)
    axes[1].set_title('Reconstructed Image')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nReconstructed image saved to: {save_path}")



## Example 2: Variable Rate Compression

The QARV model supports variable rate compression. Let's try different compression rates.



In [ ]:
import math
import numpy as np

if test_image_path.exists():
    lambdas = [16, 64, 256, 1024]
    
    fig, axes = plt.subplots(1, len(lambdas) + 1, figsize=(16, 4))
    
    original_img = Image.open(test_image_path)
    axes[0].imshow(original_img)
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    for idx, lmb in enumerate(lambdas):
        compressed_path = f'/tmp/compressed_lmb{lmb}.bits'
        
        model.compress_file(str(test_image_path), compressed_path, lmb=lmb)
        reconstructed = model.decompress_file(compressed_path)
        
        save_path = f'/tmp/reconstructed_lmb{lmb}.png'
        save_image(reconstructed, save_path)
        
        recon_img = Image.open(save_path)
        axes[idx + 1].imshow(recon_img)
        
        file_size = Path(compressed_path).stat().st_size
        bpp = (file_size * 8) / (original_img.height * original_img.width)
        
        axes[idx + 1].set_title(f'λ={lmb}\nBPP={bpp:.3f}')
        axes[idx + 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("Variable rate compression complete!")
    print("Lower λ = lower quality, smaller file size")
    print("Higher λ = higher quality, larger file size")



## Example 3: Download and Evaluate on Kodak Dataset

Let's download the Kodak test dataset and evaluate the model.



In [ ]:
import zipfile
from torch.hub import download_url_to_file
from tqdm import tqdm

def download_kodak_dataset(target_dir):
    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Downloading Kodak dataset to {target_dir}...")
    urls = [f'http://r0k.us/graphics/kodak/kodak/kodim{str(a).zfill(2)}.png' for a in range(1, 25)]
    
    for url in tqdm(urls):
        filename = Path(url).name
        filepath = target_dir / filename
        if not filepath.exists():
            download_url_to_file(url, str(filepath), progress=False)
    
    print(f"Downloaded {len(list(target_dir.glob('*.png')))} images")
    return target_dir

kodak_dir = datasets_root / 'kodak'
kodak_dir = download_kodak_dataset(kodak_dir)

from lvae.paths import known_datasets
known_datasets['kodak'] = str(kodak_dir)
print(f"\nKodak dataset path set to: {kodak_dir}")



## Example 4: Quick Evaluation

Run a quick evaluation on a few images from the Kodak dataset.



In [ ]:
from lvae.evaluation import imcoding_evaluate
import math

if kodak_dir.exists():
    print("Running evaluation on Kodak dataset...")
    print("This may take a few minutes...\n")
    
    results = imcoding_evaluate(model, 'kodak')
    
    print("\nEvaluation Results:")
    print(f"Average BPP: {results['bpp']:.4f}")
    print(f"Average PSNR: {results['psnr']:.2f} dB")
    print(f"Average MSE: {results['mse']:.6f}")
    
    if 'ssim' in results:
        print(f"Average SSIM: {results['ssim']:.4f}")
else:
    print("Kodak dataset not found. Skipping evaluation.")



## Example 5: Upload Your Own Image

Upload and compress your own image!



In [ ]:
if platform == 'colab':
    from google.colab import files
    from IPython.display import display
    
    print("Upload an image to compress:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\nProcessing {filename}...")
        
        compressed_path = f'/tmp/{Path(filename).stem}_compressed.bits'
        model.compress_file(filename, compressed_path)
        
        file_size = Path(compressed_path).stat().st_size
        original_size = Path(filename).stat().st_size
        compression_ratio = original_size / file_size
        
        print(f"Original: {original_size:,} bytes")
        print(f"Compressed: {file_size:,} bytes")
        print(f"Ratio: {compression_ratio:.2f}x")
        
        reconstructed = model.decompress_file(compressed_path)
        save_path = f'/tmp/{Path(filename).stem}_reconstructed.png'
        save_image(reconstructed, save_path)
        
        print(f"\nReconstructed image saved to: {save_path}")
        display(IPImage(save_path))
        
        files.download(save_path)
else:
    print("For Kaggle, please add your image to the input folder or use the file browser.")
    print("Then update the path below:")
    # custom_image_path = '/kaggle/input/your-image.png'
    # model.compress_file(custom_image_path, '/tmp/compressed.bits')



## Summary

You've learned how to:
1. ✅ Set up the Lossy-VAE project on Colab/Kaggle
2. ✅ Load pre-trained models
3. ✅ Compress and decompress images
4. ✅ Use variable rate compression
5. ✅ Evaluate models on test datasets

For more information, check the main README.md and model-specific documentation!

